In [ ]:
import pandas as pd
import pickle
import torch
import os
import re
import random
import csv
import jsonlines
import numpy as np
import pickle
import time
from tqdm import tqdm, trange
from sklearn.cluster import KMeans
from typing import Any, List, Sequence, Callable
from itertools import islice, zip_longest
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.cluster import MiniBatchKMeans

In [ ]:
## Download NQ Train and Dev dataset from https://ai.google.com/research/NaturalQuestions/download

In [ ]:
nq_dev = []

with open("v1.0-simplified_nq-dev-all.jsonl", "r+", encoding="utf8") as f:
    for item in jsonlines.Reader(f):
        
        arr = []
        ## question_text
        question_text = item['question_text']
        arr.append(question_text)

        tokens = []
        for i in item['document_tokens']:
            tokens.append(i['token'])
        document_text = ' '.join(tokens)
        
        ## example_id
        example_id = str(item['example_id'])
        arr.append(example_id)

        # document_text = item['document_text']
        ## long_answer
        annotation = item['annotations'][0]
        has_long_answer = annotation['long_answer']['start_token'] >= 0

        long_answers = [
            a['long_answer']
            for a in item['annotations']
            if a['long_answer']['start_token'] >= 0 and has_long_answer
        ]
        if has_long_answer:
            start_token = long_answers[0]['start_token']
            end_token = long_answers[0]['end_token']
            x = document_text.split(' ')
            long_answer = ' '.join(x[start_token:end_token])
            long_answer = re.sub('<[^<]+?>', '', long_answer).replace('\n', '').strip()
        arr.append(long_answer) if has_long_answer else arr.append('')

        # short_answer
        has_short_answer = annotation['short_answers'] or annotation['yes_no_answer'] != 'NONE'
        short_answers = [
            a['short_answers']
            for a in item['annotations']
            if a['short_answers'] and has_short_answer
        ]
        if has_short_answer and len(annotation['short_answers']) != 0:
            sa = []
            for i in short_answers[0]:
                start_token_s = i['start_token']
                end_token_s = i['end_token']
                shorta = ' '.join(x[start_token_s:end_token_s])
                sa.append(shorta)
            short_answer = '|'.join(sa)
            short_answer = re.sub('<[^<]+?>', '', short_answer).replace('\n', '').strip()
        arr.append(short_answer) if has_short_answer else arr.append('')

        ## title
        arr.append(item['document_title'])

        ## abs
        if document_text.find('<P>') != -1:
            abs_start = document_text.index('<P>')
            abs_end = document_text.index('</P>')
            abs = document_text[abs_start+3:abs_end]
        else:
            abs = ''
        arr.append(abs)

        ## content
        if document_text.rfind('</Ul>') != -1:
            final = document_text.rindex('</Ul>')
            document_text = document_text[:final]
            if document_text.rfind('</Ul>') != -1:
                final = document_text.rindex('</Ul>')
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
            else:
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
        else:
            content = document_text[abs_end+4:]
            content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
            content = re.sub(' +', ' ', content)
            arr.append(content)
        doc_tac = item['document_title'] + abs + content
        arr.append(doc_tac)
        language = 'en'
        arr.append(language)
        nq_dev.append(arr)

nq_dev_df = pd.DataFrame(nq_dev)
nq_dev_df.to_csv(r"nq_dev.tsv", sep="\t", mode = 'w', header=None, index =False)

In [ ]:
nq_train = []
with open("v1.0-simplified_simplified-nq-train.jsonl", "r+", encoding="utf8") as f:
    for item in jsonlines.Reader(f):
        ## question_text
        arr = []
        question_text = item['question_text']
        arr.append(question_text)

        ## example_id
        example_id = str(item['example_id'])
        arr.append(example_id)
        
        document_text = item['document_text']
        
        ## long_answer
        annotation = item['annotations'][0]
        has_long_answer = annotation['long_answer']['start_token'] >= 0

        long_answers = [
            a['long_answer']
            for a in item['annotations']
            if a['long_answer']['start_token'] >= 0 and has_long_answer
        ]
        if has_long_answer:
            start_token = long_answers[0]['start_token']
            end_token = long_answers[0]['end_token']
            x = document_text.split(' ')
            long_answer = ' '.join(x[start_token:end_token])
            long_answer = re.sub('<[^<]+?>', '', long_answer).replace('\n', '').strip()
        arr.append(long_answer) if has_long_answer else arr.append('')

        # short_answer
        has_short_answer = annotation['short_answers'] or annotation['yes_no_answer'] != 'NONE'
        short_answers = [
            a['short_answers']
            for a in item['annotations']
            if a['short_answers'] and has_short_answer
        ]
        if has_short_answer and len(annotation['short_answers']) != 0:
            sa = []
            for i in short_answers[0]:
                start_token_s = i['start_token']
                end_token_s = i['end_token']
                shorta = ' '.join(x[start_token_s:end_token_s])
                sa.append(shorta)
            short_answer = '|'.join(sa)
            short_answer = re.sub('<[^<]+?>', '', short_answer).replace('\n', '').strip()
        arr.append(short_answer) if has_short_answer else arr.append('')

        ## title
        if document_text.find('<H1>') != -1:
            title_start = document_text.index('<H1>')
            title_end = document_text.index('</H1>')
            title = document_text[title_start+4:title_end]
        else:
            title = ''
        arr.append(title)

        ## abs
        if document_text.find('<P>') != -1:
            abs_start = document_text.index('<P>')
            abs_end = document_text.index('</P>')
            abs = document_text[abs_start+3:abs_end]
        else:
            abs = ''
        arr.append(abs)

        ## content
        if document_text.rfind('</Ul>') != -1:
            final = document_text.rindex('</Ul>')
            document_text = document_text[:final]
            if document_text.rfind('</Ul>') != -1:
                final = document_text.rindex('</Ul>')
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
            else:
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
        else:
            content = document_text[abs_end+4:]
            content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
            content = re.sub(' +', ' ', content)
            arr.append(content)

        doc_tac = title + abs + content
        arr.append(doc_tac)

        language = 'en'
        arr.append(language)
        nq_train.append(arr)

nq_train_df = pd.DataFrame(nq_train)
nq_train_df.to_csv(r"nq_train.tsv", sep="\t", mode = 'w', header=None, index =False)

In [ ]:
## doc_tac denotes the concatenation of title, abstract and content

nq_dev = pd.read_csv('nq_dev.tsv', \
                     names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                     header=None, sep='\t')

nq_train = pd.read_csv('nq_train.tsv', \
                       names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                       header=None, sep='\t')

nq_dev['title'] = nq_dev['title'].map(lower)
nq_train['title'] = nq_train['title'].map(lower)


In [ ]:
## Mapping tool

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def lower(x):
    text = tokenizer.tokenize(x)
    id_ = tokenizer.convert_tokens_to_ids(text)
    return tokenizer.decode(id_)

In [ ]:
## Concat train doc and validation doc to obtain full document collection

nq_all_doc = nq_train.append(nq_dev)
nq_all_doc.reset_index(inplace = True)

In [ ]:
## Remove duplicated documents based on titles

nq_all_doc.drop_duplicates('title', inplace = True)
nq_all_doc.reset_index(inplace = True)

In [ ]:
## The total amount of documents : 109739

len(nq_all_doc)

In [ ]:
## Construct mapping relation

title_doc = {}
title_doc_id = {}
id_doc = {}
idx = 0
for i in range(len(nq_all_doc)):
    title_doc[nq_all_doc['title'][i]] =  nq_all_doc['doc_tac'][i]
    title_doc_id[nq_all_doc['title'][i]] = idx
    id_doc[idx] = nq_all_doc['doc_tac'][i]
    idx += 1

In [ ]:
## Construct Document Content File

train_file = open("NQ_doc_content.tsv", 'w') 

for docid in id_doc.keys():
    train_file.write('\t'.join([str(docid), '', '', id_doc[docid], '', '', 'en']) + '\n')
    train_file.flush()

In [ ]:
## Generate BERT embeddings for each document

id_doc_dict = {}
train_file = "NQ_doc_content.tsv"

with open(train_file, 'r') as f:
    for line in f.readlines():
        docid, _, _, content, _, _, _ = line.split("\t")
        id_doc_dict[docid] = content

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")


ids = list(id_doc_dict.keys())
text_list = [id_doc_dict[id_] for id_ in ids]

output_tensor = []
output_id_tensor = []
count = 0

for elem in tqdm(text_list):
    encoded_input = tokenizer(elem, max_length=512, return_tensors='pt', padding=True, truncation=True)
    output = model(**encoded_input).last_hidden_state.detach().cpu()[:, 0, :].numpy().tolist()
    output_tensor.extend(output)
    output_id_tensor.extend(ids[count])
    count += 1


In [ ]:
train_file = open("NQ_doc_content_embedding_bert_512.tsv", 'w') 

for idx, doc_tensor in enumerate(output_tensor):
    embedding = '|'.join([str(elem) for elem in doc_tensor])
    train_file.write('\t'.join([str(output_id_tensor[idx]), '', '', '', '', '', 'en', embedding]) + '\n')
    train_file.flush()

In [ ]:
## Apply Hierarchical K-Means on it to generate semantic IDs

In [ ]:
df = pd.read_csv(f'NQ_doc_content_embedding_bert_512.tsv',
                 names=['docid', 'url', 'title', 'body', 'anchor', 'click', 'language', 'vector'],
                 header=None, sep='\t').loc[:, ['docid', 'vector']]
df.drop_duplicates('docid', inplace = True)
old_id = df['docid'].tolist()
X = df['vector'].tolist()
for idx,v in enumerate(X):
    vec_str = v.split('|')
    if len(vec_str) != 768:
        print('vec dim error!')
        exit(1)
    X[idx] = [float(v) for v in vec_str]
X = np.array(X)
print(X.shape)
new_id_list = []

kmeans = KMeans(n_clusters=10, max_iter=300, n_init=100, init='k-means++', random_state=7, tol=1e-7)

mini_kmeans = MiniBatchKMeans(n_clusters=10, max_iter=300, n_init=100, init='k-means++', random_state=3,
                              batch_size=1000, reassignment_ratio=0.01, max_no_improvement=20, tol=1e-7)


def classify_recursion(x_data_pos):
    if x_data_pos.shape[0] <= 10:
        if x_data_pos.shape[0] == 1:
            return
        for idx, pos in enumerate(x_data_pos):
            new_id_list[pos].append(idx)
        return

    temp_data = np.zeros((x_data_pos.shape[0], 768))
    for idx, pos in enumerate(x_data_pos):
        temp_data[idx, :] = X[pos]

    if x_data_pos.shape[0] >= 1e3:
        pred = mini_kmeans.fit_predict(temp_data)
    else:
        pred = kmeans.fit_predict(temp_data)

    for i in range(10):
        pos_lists = []
        for id_, class_ in enumerate(pred):
            if class_ == i:
                pos_lists.append(x_data_pos[id_])
                new_id_list[x_data_pos[id_]].append(i)
        classify_recursion(np.array(pos_lists))

    return

print('Start First Clustering')
pred = mini_kmeans.fit_predict(X)
print(pred.shape)   #int 0-9 for each vector
print(mini_kmeans.n_iter_)

for class_ in pred:
    new_id_list.append([class_])

print('Start Recursively Clustering...')
for i in range(10):
    print(i, "th cluster")
    pos_lists = [];
    for id_, class_ in enumerate(pred):
        if class_ == i:
            pos_lists.append(id_)
    classify_recursion(np.array(pos_lists))

mapping = {}
for i in range(len(old_id)):
    mapping[old_id[i]] = new_id_list[i]

with open('IDMapping_NQ_bert_768_k10_c10.pkl', 'wb') as f:
    pickle.dump(mapping, f)

In [ ]:
with open('IDMapping_NQ_bert_768_k10_c10.pkl', 'rb') as f:
    kmeans_nq_doc_dict = pickle.load(f)

new_kmeans_nq_doc_dict = {}

for old_docid in kmeans_nq_doc_dict.keys():
    new_kmeans_nq_doc_dict[str(old_docid)] = ''.join(str(elem) for elem in kmeans_nq_doc_dict[old_docid])

In [ ]:
## Query Generation

def pad_sequence_to_length(
        sequence: Sequence,
        desired_length: int,
        default_value: Callable[[], Any] = lambda: 0,
        padding_on_right: bool = True,
) -> List:
    sequence = list(sequence)
    # Truncates the sequence to the desired length.
    if padding_on_right:
        padded_sequence = sequence[:desired_length]
    else:
        padded_sequence = sequence[-desired_length:]
    # Continues to pad with default_value() until we reach the desired length.
    pad_length = desired_length - len(padded_sequence)
    # This just creates the default value once, so if it's a list, and if it gets mutated
    # later, it could cause subtle bugs. But the risk there is low, and this is much faster.
    values_to_pad = [default_value()] * pad_length
    if padding_on_right:
        padded_sequence = padded_sequence + values_to_pad
    else:
        padded_sequence = values_to_pad + padded_sequence
    return padded_sequence

qg_file = open("NQ_doc_no_qg.tsv", 'w') 

for docid in id_doc.keys():
    qg_file.write('\t'.join([str(docid), new_kmeans_nq_doc_dict[str(docid)], id_doc[docid]]) + '\n')
    qg_file.flush()

device=torch.device("cuda")
##  You can also download from Hugging Face. This folder should be in the same path as the notebook.
model = AutoModelForSeq2SeqLM.from_pretrained(
    "castorini/doc2query-t5-base-msmarco"
)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model=torch.nn.DataParallel(model, device_ids=[0,1]).cuda()
    
model.eval()
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="doc2query-t5-base-msmarco"
)

## format: {Did, new_did, doc_content}
need_qg = 'NQ_doc_no_qg.tsv'
## format: {did, qg, qid(qg_{did}_i)}
qg_save = "NQ_doc_qg.tsv"

did_list = []
qg_list = []
qid_list = []
with open(qg_save, "a", newline='') as fw:

    need_qg_file = pd.read_csv(need_qg, names=['Did', 'did' ,'content'], header=None, sep='\t')

    for i in trange(len(need_qg_file)):
        next_n_lines = need_qg_file['content'][i]
        batch_input_ids = []
        sen = next_n_lines[:512]
        batch_input_ids.append(tokenizer.encode(text=sen, add_special_tokens=True))

        max_len = max([len(sen) for sen in batch_input_ids] )
        batch_input_ids = [
            pad_sequence_to_length(
                sequence=sen, desired_length=max_len, default_value=lambda : tokenizer.pad_token_id,
                padding_on_right=False
            ) for sen in batch_input_ids
        ]

        batch_input_ids = torch.tensor(data=batch_input_ids, dtype=torch.int64, device=device)

        generated = model.module.generate(
            input_ids=batch_input_ids,
            max_length=32,
            do_sample=True,
            num_return_sequences=10,
        )

        generated = tokenizer.batch_decode(sequences=generated.tolist(), skip_special_tokens=True)

        for index, g in enumerate(generated):
            fw.write('\t'.join([g, need_qg_file['Did'][i], need_qg_file['did'][i]]) + '\n')
            fw.flush()

In [ ]:
## Replace Original IDs with Semantic IDs

In [ ]:
nq_dev = pd.read_csv('nq_dev.tsv', \
                     names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                     header=None, sep='\t')

nq_train = pd.read_csv('nq_train.tsv', \
                       names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                       header=None, sep='\t')

In [ ]:
nq_train['oldid'] = nq_train['title'].map(title_doc_id)
nq_train['newid'] = nq_train['oldid'].map(new_kmeans_nq_doc_dict)
nq_train = nq_train.loc[:, ['query', 'id', 'newid']]  
nq_train.to_csv('nq_train_doc_newid.tsv', sep='\t', header=None, index=False, encoding='utf-8')

In [ ]:
nq_dev['oldid'] = nq_dev['title'].map(title_doc_id)
nq_dev['newid'] = nq_dev['oldid'].map(new_kmeans_nq_doc_dict)
nq_dev = nq_dev.loc[:, ['query', 'id', 'newid']]  
nq_dev.to_csv('nq_dev_doc_newid.tsv', sep='\t', header=None, index=False, encoding='utf-8')